Hi, i am implementing BERT however, i am getting an error of conflict of libraries and protobuf as it needs me to update the protobuf library. i cant update it right now as i am also finishing my MSc project this week which is using the old protobuf and when attempting to uppgrade, my project didnt work as it requires protobuf 3.20 whereas BERT requires higher. please excuse my submissionthis week as i am finishing my MSc project and degree program and unable to run the training process until i finish my project demonstration.i am studying MSc intelligent robotics at university of york.

In [11]:
import pandas as pd
import dask.dataframe as dd
import re
trainpanda=dd.read_csv('train_E6oV3lV.csv')
data=[]
for i in range(0,31962):

    x=trainpanda.loc[i,'tweet'].values
    f=x.compute()
    #this is cleaning a sample text extracted from the dataset using regex as requested.
    #
    cleaning = re.sub(r'[^\w\s\d#]', '', f[0])#remove anything that s not words, numbers or spaces
    #but not hashtags as they contain good information
    cleaning = re.sub(r'@\w+', '', cleaning)# removing @user (optional)
    #cleaning = re.sub(r'\s+', ' ', cleaning).strip()#removing whitespaces
    cleaning = re.sub(r'http\S+|www\S+|https\S+', '', cleaning)#removing urls
    cleaned = cleaning.encode('ascii', 'ignore').decode('ascii')#remove unicode characters
    data.append(cleaned)

In [12]:
data2=[]
def correct(word):
    # Apply the necessary correction logic to fix the word
    # Example: Replace 'q' with 'g'
    corrected_word = word.replace('user', "")   
    return corrected_word
for row in data:
    cleaner=row.replace('user', "")
    data2.append(cleaner)

In [22]:
#trainlist=trainpanda['tweet'].tolist()
#data2
#trainpanda.compute()
labels=trainpanda['label'].compute().tolist()
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [28]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import numpy as np

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Assuming binary classification (hate speech vs. non-hate speech)

# Preprocess your dataset
# Assume `tweets` is a list of strings, and `labels` is a list of corresponding labels (0 or 1)
tweets = data2
# Convert label column to a list of labels
labels = trainpanda['label'].compute().tolist()
# Assume `tweets` is a list of strings, and `labels` is a list of corresponding labels (0 or 1)
input_ids = []
attention_masks = []

for tweet in tweets:
    encoded_tweet = tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        max_length=128,  # Adjust this based on your needs
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded_tweet['input_ids'])
    attention_masks.append(encoded_tweet['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# DataLoader
batch_size = 32
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=2e-5)  # Adjust learning rate as needed
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 5  # Adjust this based on your needs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        input_ids, attention_masks, labels = batch
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}")


ModuleNotFoundError: No module named 'transformers'

In [24]:
pip install transformers

     ---------------------------------------- 7.5/7.5 MB 7.1 MB/s eta 0:00:00
     -------------------------------------- 268.8/268.8 kB 8.3 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 8.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build safetensors
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [25 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\flax.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\numpy.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\paddle.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\tensorflow.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\torch.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\__init__.py -> build\lib.win-amd64-cpython-39\safetensors
  running build_ext
  running build_rust
  error: can't find Rust compiler
  
  If you are using an outdated pip version, it is possible a prebuilt wheel is available for this package but pip is not able to install from it. Installing f

In [25]:
pip install --upgrade pip


     ---------------------------------------- 2.1/2.1 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install transformers


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     -------------------------------------- 118.5/118.5 kB 1.7 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
  Using cached tokenizers-0.13.3-cp39-cp39-win_amd64.whl (3.5 MB)
  Using cached safetensors-0.3.3.tar.gz (35 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached transformers-4.32.0-py3-

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [25 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\flax.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\numpy.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\paddle.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\tensorflow.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\torch.py -> build\lib.win-amd64-cpython-39\safetensors
  copying py_src\safetensors\__init__.py -> build\lib.win-amd64-cpython-39\safetensors
  running build_ext
  running build_rust
  error: can't find Rust compiler
  
  If you are using an outdated pip version, it is possible a prebuilt wheel is available for this package but pip is not able to install from it. Installing f